# fastgpu

> API details

In [ ]:
# default_exp utils

In [ ]:
#export
from fastgpu import *
from fastcore.all import *

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
path = Path('~/git/fastai2/nbs/examples/exps').expanduser()

In [ ]:
path.ls()

(#7) [/home/jhoward/git/fastai2/nbs/examples/exps/complete,/home/jhoward/git/fastai2/nbs/examples/exps/pending,/home/jhoward/git/fastai2/nbs/examples/exps/fail,/home/jhoward/git/fastai2/nbs/examples/exps/running,/home/jhoward/git/fastai2/nbs/examples/exps/to_run,/home/jhoward/git/fastai2/nbs/examples/exps/out,/home/jhoward/git/fastai2/nbs/examples/exps/t.sh]